# Phase-Folding on Multiple Ephemerides

This notebook loads a triple eclipsing system (KIC 2835289) containing an inner-ellipsoidal variable, flattens the light curve, and phase-folds on both orbital periods.

In [ ]:
from lightkurve import search_lightcurve
from lcviz import LCviz

lc = search_lightcurve("KIC 2835289", author="Kepler", cadence="long", quarter=[9,11,13,15,17]).download_all().stitch()

lcviz = LCviz()
lcviz.load_data(lc)
lcviz.show()

## Flattening / Detrending

Here we'll use the API to open the plugin UI, set the inputs, and flatten the light curve.  To reproduce this workflow interactively, open the "Flatten" plugin in the plugin tray (click the three horizontal bars in the top-right of lcviz), and set the corresponding inputs in the plugin UI. Modifying the parameters in the UI will produce live-previews of the detrended light curve.

In [ ]:
flatten = lcviz.plugins['Flatten']
flatten.open_in_tray()

In [ ]:
dir(flatten)

In [ ]:
# hide the preview of the flattened LC so we can inspect the trend and finetune flattening inputs
flatten.show_live_preview = False

In [ ]:
flatten.window_length = 1000
flatten.polyorder = 2
flatten.sigma = 3
flatten.niters = 3

In [ ]:
flattened_lc = flatten.flatten()

In [ ]:
flatten.close_in_tray()

## Phase-Folding

We can use the "Ephemeris" plugin to visualize two periodic features in this light curve: one period for the inner binary, and another period for the outer star's orbit.  As each ephemeris is added, a new phase viewer will be created to show the light curve folded on that period.  As with the Flatten plugin above, the UI allows for live-updating interactive workflows through the "Ephemeris" plugin.

In [ ]:
ephem = lcviz.plugins['Ephemeris']
ephem.open_in_tray()

In [ ]:
dir(ephem)

In [ ]:
ephem.rename_component('default', 'inner ELV')

In [ ]:
ephem.method = 'Lomb-Scargle'

In [ ]:
ephem.adopt_period_at_max_power()

In [ ]:
ephem.period *= 2

In [ ]:
lcviz.viewers['flux-vs-phase:inner ELV'].set_limits(y_min=0.97, y_max=1.02)

In [ ]:
ephem.add_component('outer binary')

In [ ]:
ephem.period = 750.1
ephem.t0 = 15.8

In [ ]:
ephem.wrap_at = 0.5

In [ ]:
lcviz.viewers['flux-vs-phase:outer binary'].set_limits(x_min=-0.002, x_max=0.002)

In [ ]:
ephem.close_in_tray()

Now that we have our phase-viewers, we can click and drag on the viewer tabs to re-arrange them (stacked vertically or as nested tabs, etc).  We can also use the subset tool to highlight data in any of the three viewers and see those same data points highlight in the other viewers.  

This can be particularly useful for systems like this with multiple periodic signals.  For example: we can select the data points in the "outer binary" phase viewer corresponding to the eclipse event, view where those fall in the "inner binary" phase viewer, and then inspect any other remaining outliers that don't follow the ellipsoidal variable trend.